In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# from torch_geometric.nn import GCNConv

In [2]:
class GCNConvByHand(nn.Module):
    """maps D x N to D x N"""
    def __init__(self, dim_in):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_in, bias=True)

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k, "beta_k", beta_k)
        H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(H_k, A + torch.eye(num_nodes)))
        return x

In [3]:
class GCNByHand(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels)
        self.conv2 = GCNConvByHand(in_channels)
        self.out = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        print("x after first layer\n", x)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        print("x after second layer\n", x)
        x = torch.relu(x)
        x = torch.transpose(x, 0, 1)
        x = self.out(x)
        print("x after output layer\n", x)
        x = F.softmax(x, dim=1)
        # x = torch.transpose(x, 0, 1)
        return x

In [4]:
num_nodes = 3
num_features = 2
num_labels = 5

# the following needs to be replaced by some node embeddings
x = torch.randn((num_features, num_nodes)) # D x N

# print("x", x, x.shape)

A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix ( N x N )

model = GCNByHand(in_channels=num_features, out_channels=num_labels)

out = model(x, A)

print("output")
print(out)

x after first layer
 tensor([[ 0.0752,  0.0200, -0.0188],
        [ 1.4197,  1.0271,  0.9195]], grad_fn=<AddBackward0>)
x after second layer
 tensor([[ 1.1997,  0.8579,  0.8234],
        [-0.7204, -0.4176, -0.3811]], grad_fn=<AddBackward0>)
x after output layer
 tensor([[ 0.1011,  0.9234, -0.0469, -0.4966, -0.0660],
        [ 0.1043,  0.6906, -0.0246, -0.3906,  0.1015],
        [ 0.1046,  0.6672, -0.0224, -0.3799,  0.1183]],
       grad_fn=<AddmmBackward0>)
output
tensor([[0.1807, 0.4112, 0.1558, 0.0994, 0.1529],
        [0.1893, 0.3402, 0.1664, 0.1154, 0.1887],
        [0.1899, 0.3333, 0.1673, 0.1170, 0.1925]], grad_fn=<SoftmaxBackward0>)


In [5]:
# out[2, :].sum()

In [6]:
A = np.array(A)

H_k = np.array(x)
# H_k

# H_k__A = H_k @ A
# H_k__A, H_k__A.shape

omega_0 = np.array(model.conv1.linear.weight.detach().numpy())
beta_0 = np.array(model.conv1.linear.bias.detach().numpy()).reshape((-1, 1))
# print(omega_0, beta_0)


preactivation = beta_0 @ np.ones(num_nodes).reshape((1, -1)) + omega_0 @ H_k @ (
    A + np.identity(num_nodes)
)
preactivation

array([[ 0.07516677,  0.02001623, -0.01877145],
       [ 1.41970051,  1.02709705,  0.91953606]])